In [11]:
import sys
sys.path.append("../src")

In [12]:
print(sys.path)

['/Users/lukaszbala/Documents/Python/adthena_task/notebooks', '/Users/lukaszbala/Documents/Python/adthena_task/notebooks', '/Users/lukaszbala/.local/bin', '/Users/lukaszbala/Library/Python/3.8/bin', '/Users/lukaszbala/anaconda3/lib/python38.zip', '/Users/lukaszbala/anaconda3/lib/python3.8', '/Users/lukaszbala/anaconda3/lib/python3.8/lib-dynload', '', '/Users/lukaszbala/.local/lib/python3.8/site-packages', '/Users/lukaszbala/anaconda3/lib/python3.8/site-packages', '/Users/lukaszbala/anaconda3/lib/python3.8/site-packages/aeosa', '/Users/lukaszbala/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/Users/lukaszbala/.ipython', '../src', '../src']


In [45]:
from functools import partial
import multiprocessing
import random

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import wordpunct_tokenize, word_tokenize
import string
import tensorflow as tf
from tqdm import tqdm

from adthena_task.constants import DATA_DIR_TRAIN, DATA_DIR_TEST, SEED

ModuleNotFoundError: No module named 'tensorflow'

In [14]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukaszbala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lukaszbala/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
np.random.seed(SEED)
random.seed(SEED)

In [16]:
def clean_text(text, stemmer):
    text = str(text).lower()
    tokens = [stemmer.stem(word) for word in wordpunct_tokenize(text) if word not in list(stop_words) + list(string.punctuation)]
    text = " ".join(tokens)
    return text

In [17]:
data = pd.read_csv(DATA_DIR_TRAIN, header=None)
data.columns = ["Query", "Label"]
data = utils.shuffle(data, random_state=SEED)

In [18]:
stop_words = set(stopwords.words("english"))
stemmer = SnowballStemmer(language="english")

In [24]:
clean_text_partial = partial(clean_text, stemmer=stemmer)

In [26]:
data["Clean_query"] = data["Query"].apply(clean_text_partial)

In [27]:
data_prev = data.copy()
data = data.iloc[:int(1e4), :]

In [28]:
train, test = train_test_split(data, test_size=0.2, random_state=SEED)
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r["Clean_query"]), tags=[r["Label"]]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r["Clean_query"]), tags=[r["Label"]]), axis=1)

In [39]:
train.loc[285437, "Clean_query"].split()

['recycl', 'gift', 'uk']

In [40]:
!wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
!unzip glove.6B.zip

--2021-02-10 16:47:15--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2021-02-10 16:47:15--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2021-02-10 16:47:17--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [applic

In [41]:
glove = pd.read_csv('glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove_embedding = {key: val.values for key, val in glove.T.items()}

In [44]:
def create_embedding_matrix(word_index,embedding_dict,dimension):
    embedding_matrix=np.zeros((len(word_index)+1,dimension))

    for word,index in word_index.items():
        if word in embedding_dict:
            embedding_matrix[index]=embedding_dict[word]
    return embedding_matrix

text=["The cat sat on mat","we can play with model"]

tokenizer=tf.keras.preprocessing.text.Tokenizer(split=" ")
tokenizer.fit_on_texts(text)

text_token=tokenizer.texts_to_sequences(text)

embedding_matrix=create_embedding_matrix(tokenizer.word_index,embedding_dict=glove_embedding,dimension=100)

NameError: name 'tf' is not defined

In [43]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [44]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

In [45]:
np.min(X_train)

-0.9831994

In [46]:
logreg = LogisticRegression(n_jobs=cores, C=1e5, verbose=True)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
## Confusion matrix

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
df_cm = pd.DataFrame(conf_matrix, index=logreg.classes_, columns=logreg.classes_)
sns.heatmap(df_cm,annot=True, fmt=”d”, ax=ax)
plt.ylabel(‘True label’)
plt.xlabel(‘Predicted label’)